In [1]:
import sys
sys.path.append("..")

In [2]:
from huggingface_hub import login
from huggingface_hub import whoami
import getpass

# Prompt the user for the Hugging Face token at runtime
hf_token = getpass.getpass("Enter your Hugging Face token: ")

# Log in using the provided token
login(token=hf_token)


print(whoami(token=hf_token))


/home/4283/.conda/envs/agentic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Enter your Hugging Face token:  ········


{'type': 'user', 'id': '6644f24fe7ae8316ebf3fee4', 'name': 'LizaPiya', 'fullname': 'Fahmida Liza Piya', 'isPro': False, 'avatarUrl': '/avatars/d43d60b3eba464c3f9b44c34e43b64d6.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'Agentic_FLP', 'role': 'fineGrained', 'createdAt': '2026-01-19T16:34:06.315Z', 'fineGrained': {'canReadGatedRepos': True, 'global': [], 'scoped': [{'entity': {'_id': '6644f24fe7ae8316ebf3fee4', 'type': 'user', 'name': 'LizaPiya'}, 'permissions': ['repo.content.read']}]}}}}


In [3]:
import torch
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [4]:
model_id = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
).eval()

`torch_dtype` is deprecated! Use `dtype` instead!


In [6]:
from llm_as_a_judge import llm_hallucination_evaluation, test_single_sample

csv_path = "/lustre/hl/users/4283/outputs/agenticsum/agenticsum_results_MIMIC.csv"

# Test on one sample first
test_single_sample(model, tokenizer, csv_path=csv_path)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📄 Source length:  4582 chars
📝 Summary length: 979 chars
📄 Source preview: <SEX> F <SERVICE> SURGERY <ALLERGIES> Iodine / Thallium-201 / Blue Dye / Iodine-Iodine Containing <ATTENDING> ___. <CHIEF COMPLAINT> Morbid obesity, BMI of 51 <MAJOR SURGICAL OR INVASIVE PROCEDURE> la...
📝 Summary preview: The patient was alert and oriented, with no significant abnormalities in vital signs or physical examination. The patient had a lower abdominal scar and was advised to avoid heavy lifting and abdomina...

🤖 MODEL RESPONSE:


---

**Rating:**

Hallucination: 0
Factual: 5
Complete: 3
Coherent: 4

**Explanation:**

The summary is largely accurate, but it lacks crucial details. The lack of
  Hallucination: 0
  Factual: 5
  Complete: 3
  Coherent: 4


In [7]:
# If test looks good, run full evaluation
df_judge = llm_hallucination_evaluation(model, tokenizer, csv_path=csv_path)


📄 Loaded 100 samples

🔍 Evaluating summaries for hallucinations...


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [08:20<00:00,  5.00s/it]


📊 HALLUCINATION EVALUATION RESULTS:
Hallucination (1-5)      : 1.88 ± 1.00
Factual Consistency (1-5): 4.01 ± 1.00
Completeness (1-5)       : 3.43 ± 0.79
Coherence (1-5)          : 3.88 ± 0.92

📋 ADDITIONAL INSIGHTS:
• High hallucination (≥4):        0/100
• Low factual consistency (≤2):   1/100
• Perfect factual consistency (5): 47/100
• Poor completeness (≤2):         4/100

💾 Results saved to: llama_judge_results.csv
